In [1]:
#importing the necessary libraries
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box,shape,Point,LineString,Polygon
import pandas as pd
from fiona import open
from fastkml import kml
import xml.etree.ElementTree as ET

def kml_to_gdf(kml_file):
    with open(kml_file, 'r', encoding='utf-8') as f:
        doc = f.read().encode('utf-8')
        
    k = kml.KML()
    k.from_string(doc)
    
    document = list(k.features())[0]
    
    # Extracting the first folder
    folder = list(document.features())[0]
    
    placemarks = []
    
    for feature in folder.features():
        for subfeature in feature.features():
            placemarks.append({
                'name': subfeature.name,
                'description': subfeature.description,
                'geometry': shape(subfeature.geometry)
            })
    
    gdf = gpd.GeoDataFrame(placemarks)
    return gdf

In [4]:
def parse_kml_coordinates(coord_text):
    coords = []
    for coord in coord_text.strip().split():
        lon, lat, _ = map(float, coord.split(','))
        coords.append((lon, lat))
    return coords

def kml_to_gdf(kml_file):
    tree = ET.parse(kml_file)
    root = tree.getroot()

    namespace = {'kml': 'http://www.opengis.net/kml/2.2'}

    placemarks = []

    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else None
        description = placemark.find('kml:description', namespace).text if placemark.find('kml:description', namespace) is not None else None
        geometry = None

        if placemark.find('.//kml:Point/kml:coordinates', namespace) is not None:
            coords = placemark.find('.//kml:Point/kml:coordinates', namespace).text
            geometry = Point(parse_kml_coordinates(coords)[0])
        elif placemark.find('.//kml:LineString/kml:coordinates', namespace) is not None:
            coords = placemark.find('.//kml:LineString/kml:coordinates', namespace).text
            geometry = LineString(parse_kml_coordinates(coords))
        elif placemark.find('.//kml:Polygon/kml:coordinates', namespace) is not None:
            coords = placemark.find('.//kml:Polygon/kml:coordinates', namespace).text
            geometry = Polygon(parse_kml_coordinates(coords))

        if geometry is not None:
            placemarks.append({
                'name': name,
                'description': description,
                'geometry': geometry
            })
        print(f"Number of placemarks processed: {len(placemarks)}")
    gdf = gpd.GeoDataFrame(placemarks)
    return gdf

In [5]:
gdf = kml_to_gdf(r'C:\Users\Ankit\Documents\Vedanshi\ML-hands-on\course_work\crown\data\crown.kml')
gdf

Number of placemarks processed: 0


""


In [7]:
def print_kml_structure(element, indent=''):
    print(f"{indent}{element.tag}")
    for child in element:
        print_kml_structure(child, indent + '  ')

tree = ET.parse(r'C:\Users\Ankit\Documents\Vedanshi\ML-hands-on\course_work\crown\data\crown.kml')
root = tree.getroot()
print_kml_structure(root)

{http://www.opengis.net/kml/2.2}kml
  {http://www.opengis.net/kml/2.2}Document
    {http://www.opengis.net/kml/2.2}name
    {http://www.google.com/kml/ext/2.2}CascadingStyle
      {http://www.opengis.net/kml/2.2}Style
        {http://www.opengis.net/kml/2.2}IconStyle
          {http://www.opengis.net/kml/2.2}scale
          {http://www.opengis.net/kml/2.2}Icon
            {http://www.opengis.net/kml/2.2}href
          {http://www.opengis.net/kml/2.2}hotSpot
        {http://www.opengis.net/kml/2.2}LabelStyle
        {http://www.opengis.net/kml/2.2}LineStyle
          {http://www.opengis.net/kml/2.2}color
          {http://www.opengis.net/kml/2.2}width
        {http://www.opengis.net/kml/2.2}PolyStyle
          {http://www.opengis.net/kml/2.2}color
        {http://www.opengis.net/kml/2.2}BalloonStyle
    {http://www.google.com/kml/ext/2.2}CascadingStyle
      {http://www.opengis.net/kml/2.2}Style
        {http://www.opengis.net/kml/2.2}IconStyle
          {http://www.opengis.net/kml/2.2}

# failed reading the kml